In [120]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
from openpyxl import *
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

# Web scraping

In [121]:
url_link='https://www.imdb.com/chart/top'
url=url_link
url

'https://www.imdb.com/chart/top'

In [122]:
df = pd.read_html(url, header=0)

In [123]:
df=pd.DataFrame(df[0])
df.drop(columns=['Unnamed: 0','Unnamed: 4','Your Rating'],inplace=True)
df

,Rank & Title,IMDb Rating
0,1. The Shawshank Redemption (1994),9.2
1,2. The Godfather (1972),9.2
2,3. The Dark Knight (2008),9.0
3,4. The Godfather Part II (1974),9.0
4,5. 12 Angry Men (1957),9.0
...,...,...
245,246. Dersu Uzala (1975),8.0
246,247. The Help (2011),8.0
247,248. Aladdin (1992),8.0
248,249. Gandhi (1982),8.0


In [124]:
df_sorted = df.sort_values(by='IMDb Rating',ascending=False).head(10)
df_sorted

,Rank & Title,IMDb Rating
0,1. The Shawshank Redemption (1994),9.2
1,2. The Godfather (1972),9.2
2,3. The Dark Knight (2008),9.0
3,4. The Godfather Part II (1974),9.0
4,5. 12 Angry Men (1957),9.0
5,6. Schindler's List (1993),8.9
6,7. The Lord of the Rings: The Return of the K...,8.9
7,8. Pulp Fiction (1994),8.8
8,9. The Lord of the Rings: The Fellowship of t...,8.8
9,"10. Il buono, il brutto, il cattivo (1966)",8.8


In [125]:
fig =px.bar(df_sorted,x='Rank & Title',y='IMDb Rating',color='Rank & Title',
           labels={'Rank & Title':'الاسم', 'IMDb Rating':' تقييم'},
           title='اعلى عشر افلام')
fig.show()

In [126]:
df_sorted = df.sort_values(by='IMDb Rating',ascending=True).head(10)
df_sorted

,Rank & Title,IMDb Rating
249,250. Dances with Wolves (1990),8.0
225,226. The Exorcist (1973),8.0
224,225. The Best Years of Our Lives (1946),8.0
223,224. Groundhog Day (1993),8.0
222,223. Pather Panchali (1955),8.0
221,222. The Wizard of Oz (1939),8.0
220,221. Before Sunset (2004),8.0
219,220. Into the Wild (2007),8.0
218,219. Network (1976),8.0
226,227. To Be or Not to Be (1942),8.0


In [127]:
fig =px.bar(df_sorted,x='Rank & Title',y='IMDb Rating',color='Rank & Title',
           labels={'Rank & Title':'الاسم', 'IMDb Rating':' تقييم'},
           title='اقل عشر افلام')
fig.show()

# Scrape Twitter

In [ ]:
# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(
    'كأس العالم للأندية until:2023-02-15 since:2023-01-01').get_items()):
    if i>10000:
        break
    attributes_container.append([tweet.user.username,tweet.user.displayname , tweet.date, 
                                 tweet.likeCount, tweet.sourceLabel, tweet.content, 
                                 tweet.lang,tweet.coordinates, tweet.place])
     
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Name", "Date Created", 
                                                        "Number of Likes", "Source of Tweet", "Tweet", 
                                                        "Language", "Coordinates","place"])

tweets_df["cleanTweet"] = tweets_df["Tweet"].apply(lambda s: re.sub(r'(?:\@\w+)','', s))# remove string like: @kmkk 
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(r'(?:[a-zA-Z]+)','', s))# remove english word
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(  r'([$@&!~#^*+=-_]+)'    ,'', s)) 
tweets_df = tweets_df[tweets_df['Language'] ==  "ar"]
tweets_df['Date Created'] = tweets_df['Date Created'].apply(lambda a: pd.to_datetime(a).date())
print(tweets_df)
tweets_df.to_excel('TwitterData3.xlsx')

In [149]:
df = pd.read_excel('TwitterData3.xlsx')
df.drop(columns='Unnamed: 0',inplace=True)
df

,User,Name,Date Created,Number of Likes,Source of Tweet,Tweet,Language,Coordinates,place,cleanTweet
0,AX98jfg8k94,SAMIII,2023-02-14,0,Twitter for iPhone,@iwamzyan 😂😂😂 كأس العالم للاندية هل تطن ان الم...,ar,NaN,NaN,😂😂😂 كأس العالم للاندية هل تطن ان المغرب قام ب...
1,hebasay39744927,🦅♥️🇪🇬 heba.sayed‏,2023-02-14,38,Twitter for Android,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,ar,NaN,NaN,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...
2,2oios,-ﺎلسسَآطي ‏𖤐,2023-02-14,0,Twitter for iPhone,@toldo2002 @tooothy_ وبإذن الله ان التاسعة تتح...,ar,NaN,NaN,وبإذن الله ان التاسعة تتحق السنه ذي وكاس الع...
3,mohamed90108304,Mohamed Metwally Shaarawi ( ابو بـودئ )🇪🇬♥️🇸🇦,2023-02-14,1,Twitter for iPhone,"أعلن الاتحاد الدولي لكرة القدم ""#فيفا"" تطبيق ن...",ar,NaN,NaN,"أعلن الاتحاد الدولي لكرة القدم ""فيفا"" تطبيق نظ..."
4,fstp1396,الزكرتي,2023-02-14,2,Twitter for iPhone,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,ar,NaN,NaN,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...
...,...,...,...,...,...,...,...,...,...,...
9819,aljeban,أحمد | AHMED,2023-02-12,1,Twitter for iPhone,#ريال_مدريد بطل #كأس_العالم_للأندية 2022\n ...,ar,NaN,NaN,ريالمدريد بطل كأسالعالمللأندية 2022\n ...
9820,sayyidaty,برنامج سيدتي,2023-02-12,3,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9821,YaHalaShow,برنامج ياهلا,2023-02-12,1,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9822,sa3etshabab,ساعة شباب,2023-02-12,0,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...


In [150]:
df.columns

Index(['User', 'Name', 'Date Created', 'Number of Likes', 'Source of Tweet',
       'Tweet', 'Language', 'Coordinates', 'place', 'cleanTweet'],
      dtype='object')

In [151]:
df.columns=['User', 'Name', 'Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet']

In [152]:
df

,User,Name,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language,Coordinates,place,cleanTweet
0,AX98jfg8k94,SAMIII,2023-02-14,0,Twitter for iPhone,@iwamzyan 😂😂😂 كأس العالم للاندية هل تطن ان الم...,ar,NaN,NaN,😂😂😂 كأس العالم للاندية هل تطن ان المغرب قام ب...
1,hebasay39744927,🦅♥️🇪🇬 heba.sayed‏,2023-02-14,38,Twitter for Android,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,ar,NaN,NaN,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...
2,2oios,-ﺎلسسَآطي ‏𖤐,2023-02-14,0,Twitter for iPhone,@toldo2002 @tooothy_ وبإذن الله ان التاسعة تتح...,ar,NaN,NaN,وبإذن الله ان التاسعة تتحق السنه ذي وكاس الع...
3,mohamed90108304,Mohamed Metwally Shaarawi ( ابو بـودئ )🇪🇬♥️🇸🇦,2023-02-14,1,Twitter for iPhone,"أعلن الاتحاد الدولي لكرة القدم ""#فيفا"" تطبيق ن...",ar,NaN,NaN,"أعلن الاتحاد الدولي لكرة القدم ""فيفا"" تطبيق نظ..."
4,fstp1396,الزكرتي,2023-02-14,2,Twitter for iPhone,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,ar,NaN,NaN,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...
...,...,...,...,...,...,...,...,...,...,...
9819,aljeban,أحمد | AHMED,2023-02-12,1,Twitter for iPhone,#ريال_مدريد بطل #كأس_العالم_للأندية 2022\n ...,ar,NaN,NaN,ريالمدريد بطل كأسالعالمللأندية 2022\n ...
9820,sayyidaty,برنامج سيدتي,2023-02-12,3,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9821,YaHalaShow,برنامج ياهلا,2023-02-12,1,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9822,sa3etshabab,ساعة شباب,2023-02-12,0,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...


In [153]:
df_sorted = df.sort_values(by='Number_of_Likes',ascending=False).head(10)
fig =px.bar(df_sorted,x='User',y='Number_of_Likes',hover_data=['User', 'Name','cleanTweet'],color='User',
           labels={'User':'المستخدم', 'Number_of_Likes':'عدد الاعجاب'},
           title='اعلى عشر تغريدات حازت على اعجاب الاخرين')
fig.show()

- The highest tweet was about (Saudi Arabia will host the 2023 Club World Cup 🇸🇦)

In [154]:
df['Total_of_Likes'] = df.groupby(['User']).Number_of_Likes.transform('sum')

In [155]:
df

,User,Name,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language,Coordinates,place,cleanTweet,Total_of_Likes
0,AX98jfg8k94,SAMIII,2023-02-14,0,Twitter for iPhone,@iwamzyan 😂😂😂 كأس العالم للاندية هل تطن ان الم...,ar,NaN,NaN,😂😂😂 كأس العالم للاندية هل تطن ان المغرب قام ب...,0
1,hebasay39744927,🦅♥️🇪🇬 heba.sayed‏,2023-02-14,38,Twitter for Android,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,ar,NaN,NaN,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,40
2,2oios,-ﺎلسسَآطي ‏𖤐,2023-02-14,0,Twitter for iPhone,@toldo2002 @tooothy_ وبإذن الله ان التاسعة تتح...,ar,NaN,NaN,وبإذن الله ان التاسعة تتحق السنه ذي وكاس الع...,0
3,mohamed90108304,Mohamed Metwally Shaarawi ( ابو بـودئ )🇪🇬♥️🇸🇦,2023-02-14,1,Twitter for iPhone,"أعلن الاتحاد الدولي لكرة القدم ""#فيفا"" تطبيق ن...",ar,NaN,NaN,"أعلن الاتحاد الدولي لكرة القدم ""فيفا"" تطبيق نظ...",1
4,fstp1396,الزكرتي,2023-02-14,2,Twitter for iPhone,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,ar,NaN,NaN,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,2
...,...,...,...,...,...,...,...,...,...,...,...
9819,aljeban,أحمد | AHMED,2023-02-12,1,Twitter for iPhone,#ريال_مدريد بطل #كأس_العالم_للأندية 2022\n ...,ar,NaN,NaN,ريالمدريد بطل كأسالعالمللأندية 2022\n ...,1
9820,sayyidaty,برنامج سيدتي,2023-02-12,3,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,4
9821,YaHalaShow,برنامج ياهلا,2023-02-12,1,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,37
9822,sa3etshabab,ساعة شباب,2023-02-12,0,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,2


In [156]:
df_sorted = df.sort_values(by='Total_of_Likes',ascending=False)
df_sorted.drop(columns=['Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet'],inplace=True)
df_sorted.drop_duplicates(inplace=True)
df_sorted

,User,Name,Total_of_Likes
1527,football_ll55,نواف الآسيوي 🇸🇦,8139
646,hilalstuff,أخبار الهلال,7877
8847,fifaworldcup_ar,كأس العالم FIFA 🏆,7691
634,MnbrAlhilal,منبر الهلال,5840
4764,saudiFF,الاتحاد السعودي لكرة القدم,5756
...,...,...,...
5597,azooozyy_,عز,0
5596,wmn120,maha,0
1655,Jaber00515,ابو فيصل,0
5594,ahmd88524189,احمد القحطاني,0


In [157]:
fig =px.bar(df_sorted.head(),x='Name',y='Total_of_Likes',color='Name',
           labels={'Name':'المستخدم', 'Total_of_Likes':'عدد الاعجاب'},
           title='اعلى خمس حسابات حازت على اعجاب الاخرين')
fig.show()

In [158]:
Data=pd.DataFrame(df['Source_of_Tweet'].value_counts()).head()

In [159]:
fig = px.pie(Data, values='Source_of_Tweet', names=Data.index, title='اعلى خمس مصادر للتغريدة')
fig.show()

- Top Source of Tweet is Twitter for iphone

In [ ]:
# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(
    'يوم التاسيس until:2023-02-15 since:2023-02-01').get_items()):
    if i>1000:
        break
    attributes_container.append([tweet.user.username,tweet.user.displayname , tweet.date, 
                                 tweet.likeCount, tweet.sourceLabel, tweet.content, 
                                 tweet.lang,tweet.coordinates, tweet.place])
     
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Name", "Date Created", 
                                                        "Number of Likes", "Source of Tweet", "Tweet", 
                                                        "Language", "Coordinates","place"])

tweets_df["cleanTweet"] = tweets_df["Tweet"].apply(lambda s: re.sub(r'(?:\@\w+)','', s))# remove string like: @kmkk 
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(r'(?:[a-zA-Z]+)','', s))# remove english word
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(  r'([$@&!~#^*+=-_]+)'    ,'', s)) 
tweets_df = tweets_df[tweets_df['Language'] ==  "ar"]
tweets_df['Date Created'] = tweets_df['Date Created'].apply(lambda a: pd.to_datetime(a).date())
print(tweets_df)
tweets_df.to_excel('TwitterData2.xlsx')

In [160]:
df = pd.read_excel('TwitterData2.xlsx')
df.columns

Index(['Unnamed: 0', 'User', 'Name', 'Date Created', 'Number of Likes',
       'Source of Tweet', 'Tweet', 'Language', 'Coordinates', 'place',
       'cleanTweet'],
      dtype='object')

In [161]:
df

,Unnamed: 0,User,Name,Date Created,Number of Likes,Source of Tweet,Tweet,Language,Coordinates,place,cleanTweet
0,0,askc3r,اسأل عن سيارتك,2023-02-14,1,Twitter for iPhone,🎈✉️من الـخـاص✉️🎈 \n\nسلام عليكم ، وش افضل الش...,ar,NaN,NaN,🎈✉️من الـخـاص✉️🎈 \n\nسلام عليكم ، وش افضل الش...
1,1,no44il,سُكرة🤍,2023-02-14,0,Twitter for iPhone,@AzizbagBag يوم التاسيس السعودي تختصر \nمشاعر ...,ar,NaN,NaN,يوم التاسيس السعودي تختصر \nمشاعر الحبّ والان...
2,2,masdarak1,عبدالرحمن,2023-02-14,0,Twitter for Android,@najeebauto الجواب \nيرمز الى أصول الدولة السع...,ar,NaN,NaN,الجواب \nيرمز الى أصول الدولة السعودية بعراقت...
3,3,MKKX_407,👑♥️.NOUFAH.👑♥️,2023-02-14,0,Twitter for iPhone,@WoC1h ٧ ملوك 🇸🇦💗\n\n.\n#مسابقة_يوم_التأسيس,ar,NaN,NaN,٧ ملوك 🇸🇦💗\n\n.\nمسابقةيومالتأسيس
4,4,Malak3234,ملاك الشهري,2023-02-14,0,Twitter for iPhone,@smco_sa #يوم_التاسيس \n\nالخدمات الإنسانية ال...,ar,NaN,NaN,يومالتاسيس \n\nالخدمات الإنسانية التي تقدمها ...
...,...,...,...,...,...,...,...,...,...,...,...
886,995,Officially40,📜,2023-02-14,0,Twitter for iPhone,وين يوم التأسيس مجهزه فديو كليب وانا امشي على ...,ar,NaN,NaN,وين يوم التأسيس مجهزه فديو كليب وانا امشي على ...
887,996,Moe_mkh_07_0009,مدرسة إبتدائية غامد للبنات,2023-02-14,0,Twitter for iPhone,من شرقها لغربها فنون تنوعت وعكست ثقافة وهوية.....,ar,NaN,NaN,من شرقها لغربها فنون تنوعت وعكست ثقافة وهوية.....
888,997,MOE_ARP_08_0091,المتوسطة الثالثة للبنات بأبها,2023-02-14,0,Twitter for Android,شاركت الإبتدائيه و المتوسطة الثالثة للبنات بأب...,ar,NaN,NaN,شاركت الإبتدائيه و المتوسطة الثالثة للبنات بأب...
889,999,MOE_ARP_09_0095,متوسطة الرونه الأولى,2023-02-14,0,Twitter for iPhone,مشاركات طالبات الصف الثاني متوسط تحت اشراف رائ...,ar,NaN,NaN,مشاركات طالبات الصف الثاني متوسط تحت اشراف رائ...


In [162]:
df=pd.DataFrame(df)
df.drop(columns='Unnamed: 0',inplace=True)

In [163]:
df.columns=['User', 'Name', 'Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet']

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   User             891 non-null    object        
 1   Name             891 non-null    object        
 2   Date_Created     891 non-null    datetime64[ns]
 3   Number_of_Likes  891 non-null    int64         
 4   Source_of_Tweet  891 non-null    object        
 5   Tweet            891 non-null    object        
 6   Language         891 non-null    object        
 7   Coordinates      13 non-null     object        
 8   place            13 non-null     object        
 9   cleanTweet       891 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 69.7+ KB


In [165]:
df_sorted = df.sort_values(by='Number_of_Likes',ascending=False).head(10)
fig =px.bar(df_sorted,x='User',y='Number_of_Likes',hover_data=['User', 'Name','cleanTweet'],color='User',
           labels={'User':'المستخدم', 'Number_of_Likes':'عدد الاعجاب'},
           title='اعلى عشر تغريدات حازت على اعجاب الاخرين')
fig.show()

In [166]:
Data=pd.DataFrame(df['Source_of_Tweet'].value_counts()).head()

In [167]:
fig = px.pie(Data, values='Source_of_Tweet', names=Data.index, title='اعلى خمس مصادر للتغريدة')
fig.show()

In [170]:
df['Total_of_Likes'] = df.groupby(['User']).Number_of_Likes.transform('sum')

In [171]:
df_sorted = df.sort_values(by='Total_of_Likes',ascending=False)
df_sorted.drop(columns=['Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet'],inplace=True)
df_sorted.drop_duplicates(inplace=True)
df_sorted

,User,Name,Total_of_Likes
181,umwail99,umwael,189
791,Fksa40,░ځـٌٌـٌٌالُـِـِِـِِِـِِـِـڊ البل ░ k̯͡s̯͡a̯͡,161
395,Nawal_raiyes,نوال الريس🇸🇦👑👑١٧٢٧م🐪,48
642,Trend_ksu,online store,43
336,Culture821978,فرجت🌱,33
...,...,...,...
378,MOE_YNB_01_0008,مدرسة النجف للطفولة المبكرة,0
379,MOE_ARP_09_0017,الابتدائية الخامسةوالعشرون للطفولة المبكرة بال...,0
382,mojazalanbaa,موجز الأنباء,0
385,OVo0NCJ4tPquSR2,ياسر سعد,0


In [173]:
fig =px.bar(df_sorted.head(),x='User',y='Total_of_Likes',color='User',
           labels={'User':'المستخدم', 'Total_of_Likes':'عدد الاعجاب'},
           title='اعلى خمس حسابات حازت على اعجاب الاخرين')
fig.show()